In [29]:
import bioread
import np
import pandas as pd
import heartpy as hp
import neurokit2 as nk

acq_file = bioread.read_file(r'C:\Users\User\Downloads\80_28.1.24 (NPU_template).acq')

channels_to_extract = ['ECG - ECG100C', 'C5 - Expression' ]
data = {}
for channel_name in channels_to_extract:
    channel = acq_file.named_channels[channel_name]
    data[channel_name] = channel.data

df = pd.DataFrame(data)
print(df)


         ECG - ECG100C  C5 - Expression
0             0.000000              0.0
1             0.000153              0.0
2            -0.000153              0.0
3             0.000000              0.0
4             0.000000              0.0
...                ...              ...
3132788      -0.097656              0.0
3132789      -0.100098              0.0
3132790      -0.102539              0.0
3132791      -0.105438              0.0
3132792      -0.109253              0.0

[3132793 rows x 2 columns]


In [30]:
df = df.dropna()
df = df.reset_index(drop=True)


In [31]:
non_zero_indices = df.index[df['C5 - Expression'] != 0]
index_rows_to_change = []
for i in range(len(non_zero_indices)-1):
    if non_zero_indices[i+1] == non_zero_indices[i]+1:
        index_rows_to_change.append(non_zero_indices[i])
for index in index_rows_to_change:
    df.loc[index, 'C5 - Expression'] = 0

In [32]:
print(df)

         ECG - ECG100C  C5 - Expression
0             0.000000              0.0
1             0.000153              0.0
2            -0.000153              0.0
3             0.000000              0.0
4             0.000000              0.0
...                ...              ...
3132788      -0.097656              0.0
3132789      -0.100098              0.0
3132790      -0.102539              0.0
3132791      -0.105438              0.0
3132792      -0.109253              0.0

[3132793 rows x 2 columns]


In [33]:
df_baseline = df['ECG - ECG100C'][df['C5 - Expression']==99]

print(df_baseline)

481122    0.016327
Name: ECG - ECG100C, dtype: float64


In [34]:
df_baseline_minute = pd.DataFrame()
for index in df_baseline.index:
    df_slice = df['ECG - ECG100C'].iloc[index : index + 2000 * 60]
    df_baseline_minute = pd.concat([df_baseline_minute, df_slice])
print(df_baseline_minute)

               0
481122  0.016327
481123  0.016632
481124  0.017548
481125  0.017548
481126  0.018463
...          ...
601117 -0.069733
601118 -0.070953
601119 -0.071259
601120 -0.072021
601121 -0.072632

[120000 rows x 1 columns]


In [39]:
baseline_minute_array = np.array(df_baseline_minute)
print(baseline_minute_array)

[[ 0.0163269 ]
 [ 0.01663208]
 [ 0.01754761]
 ...
 [-0.07125854]
 [-0.07202148]
 [-0.07263184]]


In [38]:

signals = nk.ecg_process(baseline_minute_array, 2000)
heart_rate = nk.ecg_rate(signals)

ValueError: Per-column arrays must each be 1-dimensional

In [ ]:
ecg_data =  {
    "n_cue": [],
    "n_no_cue": [],
    "p_cue": [],
    "p_no_cue": [],
    "u_cue": [],
    "u_no_cue": []
}
for index, row in df.iterrows():
    value = row['C5 - Expression']
    ecg_value = row['ECG - ECG100C']
    
    
    if value == 020.0:
        ecg_data["n_cue"].append(ecg_value)
    if value == 030.0:
        ecg_data["n_no_cue"].append(ecg_value)
    if value == 120.0:
        ecg_data["p_cue"].append(ecg_value)
    if value == 130.0:
        ecg_data["p_no_cue"].append(ecg_value)
    if value == 220.0:
        ecg_data["u_cue"].append(ecg_value)
    if value == 230.0:
        ecg_data["u_no_cue"].append(ecg_value)
        
max_length = max(len(ecg_data[key]) for key in ecg_data)
for key in ecg_data:
    while len(ecg_data[key]) < max_length:
        ecg_data[key].append(None)
df_ecg = pd.DataFrame(ecg_data)
print(df_ecg)


    
         


In [ ]:
## trying with HEARTPY df
ecg_sampling_rate = 2000
mean_heart_rate = {}
working_data, measures = hp.process(df["ECG - ECG100C"].values, ecg_sampling_rate)
mean_heart_rate["ECG - ECG100C"] = measures['bpm']



In [ ]:
measures

In [ ]:
## trying with HEARTPY
import numpy as np 

ecg_sampling_rate = 2000
mean_heart_rate = {}
for column in df_ecg.columns:
    nmpy = np.array(df_ecg[column])
    working_data, measures = hp.process(nmpy, ecg_sampling_rate)
    mean_heart_rate[column] = measures['bpm']
    mean_heart_rate = {column: sum(heart_rates) / len(heart_rates) for column, heart_rates in mean_heart_rate.items()}
print("Mean Heart Rates:")
for column, mean_hr in mean_heart_rate.items():
    print(f"{column}: {mean_hr} bpm")

In [ ]:


## trying with neurokit   
ecg_sampling_rate = 2000
heart_rate_data = {}
for column in df_ecg.columns:
    signals = nk.ecg_process(df_ecg[column], ecg_sampling_rate)
    heart_rate = nk.ecg_rate(signals)
    heart_rate_data[column] = heart_rate
print("Heart Rates:")
for column, heart_rate in heart_rate_data.items():
    print(f"{column}: {heart_rate.mean()} bpm")

heart_rate_df = pd.DataFrame(heart_rate_data)

mean_heart_rate_per_column = heart_rate_df.mean(axis=0)

print("Mean Heart Rate per column:")
print(mean_heart_rate_per_column)
    
    







    

In [ ]:
nmpy